Capstone Project - Recipe/Meal Recommendation System

In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import re
import numpy as np
import pickle
from nltk.tokenize import RegexpTokenizer, word_tokenize
from nltk.stem import WordNetLemmatizer
from surprise.model_selection import train_test_split

In [2]:
user_data = pd.read_csv('data/RAW_interactions.csv')
user_data.head()

,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall..."
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin..."


In [3]:
recipe_data = pd.read_csv('data/RAW_recipes.csv')
recipe_data.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [4]:
recipe_df = recipe_data.drop(columns=['contributor_id', 'submitted', 'tags', 'nutrition', 'description', 'ingredients', 'n_steps', 'steps', 'n_ingredients'])
recipe_df.head()

,name,id,minutes
0,arriba baked winter squash mexican style,137739,55
1,a bit different breakfast pizza,31490,30
2,all in the kitchen chili,112140,130
3,alouette potatoes,59389,45
4,amish tomato ketchup for canning,44061,190


In [5]:
from surprise import Reader, Dataset
# read in values as Surprise dataset 

reader = Reader()
recipe_df = Dataset.load_from_df(recipe_df, reader)


In [6]:
print(recipe_df)

In [7]:
recipes_PP = pd.read_csv('data/PP_recipes.csv')
recipes_PP.head()

,id,i,name_tokens,ingredient_tokens,steps_tokens,techniques,calorie_level,ingredient_ids
0,424415,23,"[40480, 37229, 2911, 1019, 249, 6878, 6878, 28...","[[2911, 1019, 249, 6878], [1353], [6953], [153...","[40480, 40482, 21662, 481, 6878, 500, 246, 161...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[389, 7655, 6270, 1527, 3406]"
1,146223,96900,"[40480, 18376, 7056, 246, 1531, 2032, 40481]","[[17918], [25916], [2507, 6444], [8467, 1179],...","[40480, 40482, 729, 2525, 10906, 485, 43, 8393...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[2683, 4969, 800, 5298, 840, 2499, 6632, 7022,..."
2,312329,120056,"[40480, 21044, 16954, 8294, 556, 10837, 40481]","[[5867, 24176], [1353], [6953], [1301, 11332],...","[40480, 40482, 8240, 481, 24176, 296, 1353, 66...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",1,"[1257, 7655, 6270, 590, 5024, 1119, 4883, 6696..."
3,74301,168258,"[40480, 10025, 31156, 40481]","[[1270, 1645, 28447], [21601], [27952, 29471, ...","[40480, 40482, 5539, 21601, 1073, 903, 2324, 4...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,"[7940, 3609, 7060, 6265, 1170, 6654, 5003, 3561]"
4,76272,109030,"[40480, 17841, 252, 782, 2373, 1641, 2373, 252...","[[1430, 11434], [1430, 17027], [1615, 23, 695,...","[40480, 40482, 14046, 1430, 11434, 488, 17027,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[3484, 6324, 7594, 243]"


In [8]:
users_PP = pd.read_csv('data/PP_users.csv')
users_PP.head()

,u,techniques,items,n_items,ratings,n_ratings
0,0,"[8, 0, 0, 5, 6, 0, 0, 1, 0, 9, 1, 0, 0, 0, 1, ...","[1118, 27680, 32541, 137353, 16428, 28815, 658...",31,"[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 4.0, ...",31
1,1,"[11, 0, 0, 2, 12, 0, 0, 0, 0, 14, 5, 0, 0, 0, ...","[122140, 77036, 156817, 76957, 68818, 155600, ...",39,"[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...",39
2,2,"[13, 0, 0, 7, 5, 0, 1, 2, 1, 11, 0, 1, 0, 0, 1...","[168054, 87218, 35731, 1, 20475, 9039, 124834,...",27,"[3.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 5.0, ...",27
3,3,"[498, 13, 4, 218, 376, 3, 2, 33, 16, 591, 10, ...","[163193, 156352, 102888, 19914, 169438, 55772,...",1513,"[5.0, 5.0, 5.0, 5.0, 4.0, 4.0, 5.0, 5.0, 5.0, ...",1513
4,4,"[161, 1, 1, 86, 93, 0, 0, 11, 2, 141, 0, 16, 0...","[72857, 38652, 160427, 55772, 119999, 141777, ...",376,"[5.0, 5.0, 5.0, 5.0, 4.0, 4.0, 5.0, 4.0, 5.0, ...",376


In [9]:
users_PP.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25076 entries, 0 to 25075
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   u           25076 non-null  int64 
 1   techniques  25076 non-null  object
 2   items       25076 non-null  object
 3   n_items     25076 non-null  int64 
 4   ratings     25076 non-null  object
 5   n_ratings   25076 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 1.1+ MB


In [10]:
#users_PP['ratings'] = users_PP['ratings'].astype(float)
user_df = user_data.drop(columns=['date', 'review'])
#users_PP = users_PP.rename(columns = {'u': 'user', 'items':'recipes'})
user_df.head()

,user_id,recipe_id,rating
0,38094,40893,4
1,1293707,40893,5
2,8937,44394,4
3,126440,85009,5
4,57222,85009,5


In [11]:
user_df = Dataset.load_from_df(user_df, reader)
user_df

In [12]:
pickled_map = pd.read_pickle('data/ingr_map.pkl')
pickled_map

,raw_ingr,raw_words,processed,len_proc,replaced,count,id
0,"medium heads bibb or red leaf lettuce, washed,...",13,"medium heads bibb or red leaf lettuce, washed,...",73,lettuce,4507,4308
1,mixed baby lettuces and spring greens,6,mixed baby lettuces and spring green,36,lettuce,4507,4308
2,romaine lettuce leaf,3,romaine lettuce leaf,20,lettuce,4507,4308
3,iceberg lettuce leaf,3,iceberg lettuce leaf,20,lettuce,4507,4308
4,red romaine lettuce,3,red romaine lettuce,19,lettuce,4507,4308
...,...,...,...,...,...,...,...
11654,soybeans,1,soybean,7,soybean,31,6702
11655,goose,1,goose,5,goose,8,3318
11656,ajwain,1,ajwain,6,ajwain,13,47
11657,brinjals,1,brinjal,7,brinjal,2,750


## Data Exploration

In [13]:
user_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1132367 entries, 0 to 1132366
Data columns (total 5 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   user_id    1132367 non-null  int64 
 1   recipe_id  1132367 non-null  int64 
 2   date       1132367 non-null  object
 3   rating     1132367 non-null  int64 
 4   review     1132198 non-null  object
dtypes: int64(3), object(2)
memory usage: 43.2+ MB


In [14]:
print(len(user_data['user_id'].unique()))

226570


There are 226,570 unique users in this dataset

In [15]:
recipe_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231637 entries, 0 to 231636
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   name            231636 non-null  object
 1   id              231637 non-null  int64 
 2   minutes         231637 non-null  int64 
 3   contributor_id  231637 non-null  int64 
 4   submitted       231637 non-null  object
 5   tags            231637 non-null  object
 6   nutrition       231637 non-null  object
 7   n_steps         231637 non-null  int64 
 8   steps           231637 non-null  object
 9   description     226658 non-null  object
 10  ingredients     231637 non-null  object
 11  n_ingredients   231637 non-null  int64 
dtypes: int64(5), object(7)
memory usage: 21.2+ MB


In [16]:
recipe_data = recipe_data.drop(columns=['contributor_id', 'submitted', 'nutrition', 'steps'])

In [17]:
print(len(recipe_data['id'].unique()))

231637


There are 231,637 unique recipes in this dataset.

With 226,570 users and 231,637 recipes, there are less users than there are recipes. Therefore, it is probably best for our recommender system to be user-user based.

## Preprocessing

The text data in description needs to be cleaned to ensure all punctuation is removed and words are all lowercase. The text data in the other columns looks pretty well cleaned already, but we still need to remove stopwords.

In [18]:
# Creating a function to perform cleaning steps at once
stopwords_list = stopwords.words('english')

no_bad_chars = re.compile('[!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n - ]')
no_nums = re.compile('[\d-]')

def clean_text(text):
    text = no_bad_chars.sub(' ', text) 
    text = text.lower() 
    text = ' '.join(word for word in text.split() if word not in stopwords_list)
    return text


In [19]:
#recipe_data['tags'] = recipe_data['tags'].astype(str)
tags_cleaned = recipe_data['tags'].apply(clean_text)
tags_cleaned

0         '60 minutes less' 'time make' 'course' 'main i...
1         '30 minutes less' 'time make' 'course' 'main i...
2         'time make' 'course' 'preparation' 'main dish'...
3         '60 minutes less' 'time make' 'course' 'main i...
4         'weeknight' 'time make' 'course' 'main ingredi...
                                ...                        
231632    'ham' '60 minutes less' 'time make' 'course' '...
231633    '15 minutes less' 'time make' 'course' 'prepar...
231634    '60 minutes less' 'time make' 'course' 'main i...
231635    '30 minutes less' 'time make' 'course' 'prepar...
231636    '30 minutes less' 'time make' 'course' 'prepar...
Name: tags, Length: 231637, dtype: object

In [20]:
GF = []
for row in recipe_data['tags']: 
    if "gluten-free" in row : GF.append(1)
    elif "gluten free" in row : GF.append(1) 
    else: GF.append(0) 

In [21]:
recipe_data['GF'] = GF

In [22]:
recipe_data['GF'].value_counts()

0    225894
1      5743
Name: GF, dtype: int64

In [23]:
#Ingredient lists for diet filtering:
meat = ['ham', 'beef', 'meat', 'chicken', 'pork', 'bacon', 'sausage', 'lamb', 'veal', 'turkey', 'steak', 'rib', 'frankfurter', 'duck', 'poultry', 'goat', 'liver', 'hen', 'quail', 'brisket', 'goose']
seafood = ['fish', 'shrimp', 'seafood', 'crab', 'lobster', 'clam', 'oyster', 'scallop', 'mussel', 'cod', 'salmon', 'halibut', 'shellfish', 'roe', 'tuna', 'caviar', 'pollock', 'yellowtail', 'squid', 'calamari', 'octopus', 'crawfish', 'crayfish', 'sardine', 'trout', 'flounder', 'anchovy', 'bass', 'haddock', 'sole' ]
animal_other = ['egg', 'honey']
dairy = ['milk', 'cheese', 'yogurt', 'mayonnaise', 'butter', 'margarine', 'cream']
gluten_free = ['gluten free', 'gluten-free']


vegan = ['ham', 'beef', 'meat', 'chicken', 'pork', 'bacon', 'sausage', 'lamb', 'veal', 'turkey', 'steak', 'rib', 'frankfurter', 'duck', 'poultry', 'goat', 'liver', 'hen', 'quail', 'brisket', 'goose','fish', 'shrimp', 'seafood', 'crab', 'lobster', 'clam', 'oyster', 'scallop', 'mussel', 'cod', 'salmon', 'halibut', 'shellfish', 'roe', 'tuna', 'caviar', 'pollock', 'yellowtail', 'squid', 'calamari', 'octopus', 'crawfish', 'crayfish', 'sardine', 'trout', 'flounder', 'anchovy', 'bass', 'haddock', 'sole','egg', 'honey','milk', 'cheese', 'yogurt', 'mayonnaise', 'butter', 'margarine', 'cream']

vegetarian = ['ham', 'beef', 'meat', 'chicken', 'pork', 'bacon', 'sausage', 'lamb', 'veal', 'turkey', 'steak', 'rib', 'frankfurter', 'duck', 'poultry', 'goat', 'liver', 'hen', 'quail', 'brisket', 'goose','fish', 'shrimp', 'seafood', 'crab', 'lobster', 'clam', 'oyster', 'scallop', 'mussel', 'cod', 'salmon', 'halibut', 'shellfish', 'roe', 'tuna', 'caviar', 'pollock', 'yellowtail', 'squid', 'calamari', 'octopus', 'crawfish', 'crayfish', 'sardine', 'trout', 'flounder', 'anchovy', 'bass', 'haddock', 'sole']

In [24]:
basic_token_pattern = r"(?u)\b\w\w+\b"
tokenizer = RegexpTokenizer(basic_token_pattern)

In [25]:
recipe_data['vegetarian'] = None
recipe_data['vegan'] = None



In [26]:
#vege_pattern = '|'.join(vegetarian)
#vegan_pattern = '|'.join(vegan)


#recipe_data.vegetarian = (recipe_data[~recipe_data.ingredients.str.contains(vege_pattern)])
#recipe_data.vegan = (recipe_data[~recipe_data.ingredients.str.contains(vegan_pattern)])

In [27]:
vege_pattern = '|'.join(vegetarian)
vegan_pattern = '|'.join(vegan)


recipe_data.vegetarian = recipe_data.ingredients.str.contains(vege_pattern)
recipe_data.vegan = recipe_data.ingredients.str.contains(vegan_pattern)

In [28]:
recipe_data.head()

,name,id,minutes,tags,n_steps,description,ingredients,n_ingredients,GF,vegetarian,vegan
0,arriba baked winter squash mexican style,137739,55,"['60-minutes-or-less', 'time-to-make', 'course...",11,autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,0,False,True
1,a bit different breakfast pizza,31490,30,"['30-minutes-or-less', 'time-to-make', 'course...",9,this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,0,True,True
2,all in the kitchen chili,112140,130,"['time-to-make', 'course', 'preparation', 'mai...",6,this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13,0,True,True
3,alouette potatoes,59389,45,"['60-minutes-or-less', 'time-to-make', 'course...",11,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11,0,False,True
4,amish tomato ketchup for canning,44061,190,"['weeknight', 'time-to-make', 'course', 'main-...",5,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8,0,False,False


In [29]:
new_vals = {'False': 1, 'True': 0}

In [30]:
recipe_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231637 entries, 0 to 231636
Data columns (total 11 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   name           231636 non-null  object
 1   id             231637 non-null  int64 
 2   minutes        231637 non-null  int64 
 3   tags           231637 non-null  object
 4   n_steps        231637 non-null  int64 
 5   description    226658 non-null  object
 6   ingredients    231637 non-null  object
 7   n_ingredients  231637 non-null  int64 
 8   GF             231637 non-null  int64 
 9   vegetarian     231637 non-null  bool  
 10  vegan          231637 non-null  bool  
dtypes: bool(2), int64(5), object(4)
memory usage: 16.3+ MB


In [31]:
recipe_data['vegetarian'] = recipe_data['vegetarian'].astype(str)
recipe_data['vegetarian'] = recipe_data['vegetarian'].replace({'False': 1, 'True': 0})

In [32]:
recipe_data['vegan'] = recipe_data['vegan'].astype(str)
recipe_data['vegan'] = recipe_data['vegan'].replace({'False': 1, 'True': 0})

In [33]:
recipe_data.head()

,name,id,minutes,tags,n_steps,description,ingredients,n_ingredients,GF,vegetarian,vegan
0,arriba baked winter squash mexican style,137739,55,"['60-minutes-or-less', 'time-to-make', 'course...",11,autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,0,1,0
1,a bit different breakfast pizza,31490,30,"['30-minutes-or-less', 'time-to-make', 'course...",9,this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,0,0,0
2,all in the kitchen chili,112140,130,"['time-to-make', 'course', 'preparation', 'mai...",6,this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13,0,0,0
3,alouette potatoes,59389,45,"['60-minutes-or-less', 'time-to-make', 'course...",11,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11,0,1,0
4,amish tomato ketchup for canning,44061,190,"['weeknight', 'time-to-make', 'course', 'main-...",5,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8,0,1,1


In [34]:
recipe_data['vegetarian'].value_counts()

1    134596
0     97041
Name: vegetarian, dtype: int64

In [35]:
recipe_data['vegan'].value_counts()

0    197961
1     33676
Name: vegan, dtype: int64

In [36]:
#Making column names match and merging dfs to classify user diets based on the recipes they've used
recipe_data = recipe_data.rename(columns = {'id': 'recipe_id'})

user_diets = pd.merge(user_data, recipe_data, on='recipe_id', how='left')



In [37]:
user_ratings = user_diets.copy()
user_ratings = user_ratings.drop(columns=['date', 'review', 'name', 'minutes', 'tags', 'n_steps', 'description', 'ingredients', 'n_ingredients'])
user_ratings.head()

,user_id,recipe_id,rating,GF,vegetarian,vegan
0,38094,40893,4,0,0,0
1,1293707,40893,5,0,0,0
2,8937,44394,4,0,1,0
3,126440,85009,5,0,0,0
4,57222,85009,5,0,0,0


In [38]:
user_diets = user_diets.drop(columns=['date', 'review', 'rating', 'name', 'minutes', 'tags', 'n_steps', 'description', 'ingredients', 'n_ingredients'])
user_diets.head()

,user_id,recipe_id,GF,vegetarian,vegan
0,38094,40893,0,0,0
1,1293707,40893,0,0,0
2,8937,44394,0,1,0
3,126440,85009,0,0,0
4,57222,85009,0,0,0


In [39]:
user_diets_count = user_diets.groupby(['user_id'])['recipe_id'].count()
user_diets_count.head()

user_id
1533    128
1535    794
1581      1
1634     60
1676     31
Name: recipe_id, dtype: int64

In [40]:
user_diets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1132367 entries, 0 to 1132366
Data columns (total 5 columns):
 #   Column      Non-Null Count    Dtype
---  ------      --------------    -----
 0   user_id     1132367 non-null  int64
 1   recipe_id   1132367 non-null  int64
 2   GF          1132367 non-null  int64
 3   vegetarian  1132367 non-null  int64
 4   vegan       1132367 non-null  int64
dtypes: int64(5)
memory usage: 51.8 MB


In [41]:
len(user_diets['user_id'].unique())

226570

In [42]:
user_diets = (user_diets.groupby(['user_id']).agg({'recipe_id': lambda x: x.tolist(), 'GF': sum, 'vegetarian':sum, 'vegan': sum}).reset_index())
user_diets.head()

,user_id,recipe_id,GF,vegetarian,vegan
0,1533,"[116345, 32907, 14750, 24136, 63598, 83375, 35...",11,57,8
1,1535,"[349022, 50022, 78834, 47474, 230720, 14111, 8...",20,630,114
2,1581,[341050],0,0,0
3,1634,"[16512, 158215, 34533, 44459, 26212, 168194, 3...",3,35,9
4,1676,"[34233, 99156, 66799, 166273, 250600, 179836, ...",3,4,2


In [43]:
user_diets['is_vegetarian'] = None
user_diets['is_vegan'] = None
user_diets['is_GF'] = None
user_diets.head()

,user_id,recipe_id,GF,vegetarian,vegan,is_vegetarian,is_vegan,is_GF
0,1533,"[116345, 32907, 14750, 24136, 63598, 83375, 35...",11,57,8,None,None,None
1,1535,"[349022, 50022, 78834, 47474, 230720, 14111, 8...",20,630,114,None,None,None
2,1581,[341050],0,0,0,None,None,None
3,1634,"[16512, 158215, 34533, 44459, 26212, 168194, 3...",3,35,9,None,None,None
4,1676,"[34233, 99156, 66799, 166273, 250600, 179836, ...",3,4,2,None,None,None


In [44]:
count = 0
for str_list in user_diets['recipe_id']:
    for name in str_list:
        if name == "arts":
            count += 1

print(count)

0


In [45]:
user_diets['recipe_totals'] = user_diets['recipe_id'].str.len()
#df.count(axis='columns')

In [46]:
user_diets

,user_id,recipe_id,GF,vegetarian,vegan,is_vegetarian,is_vegan,is_GF,recipe_totals
0,1533,"[116345, 32907, 14750, 24136, 63598, 83375, 35...",11,57,8,None,None,None,128
1,1535,"[349022, 50022, 78834, 47474, 230720, 14111, 8...",20,630,114,None,None,None,794
2,1581,[341050],0,0,0,None,None,None,1
3,1634,"[16512, 158215, 34533, 44459, 26212, 168194, 3...",3,35,9,None,None,None,60
4,1676,"[34233, 99156, 66799, 166273, 250600, 179836, ...",3,4,2,None,None,None,31
...,...,...,...,...,...,...,...,...,...
226565,2002371755,[393600],0,1,0,None,None,None,1
226566,2002371792,[431399],0,1,0,None,None,None,1
226567,2002371843,[516662],0,1,0,None,None,None,1
226568,2002372464,[491512],0,1,0,None,None,None,1


In [47]:
user_diets['is_vegetarian'] = np.where(user_diets['vegetarian'] >= ((user_diets['recipe_totals'])*(.75)), True, False)
user_diets.head()

,user_id,recipe_id,GF,vegetarian,vegan,is_vegetarian,is_vegan,is_GF,recipe_totals
0,1533,"[116345, 32907, 14750, 24136, 63598, 83375, 35...",11,57,8,False,None,None,128
1,1535,"[349022, 50022, 78834, 47474, 230720, 14111, 8...",20,630,114,True,None,None,794
2,1581,[341050],0,0,0,False,None,None,1
3,1634,"[16512, 158215, 34533, 44459, 26212, 168194, 3...",3,35,9,False,None,None,60
4,1676,"[34233, 99156, 66799, 166273, 250600, 179836, ...",3,4,2,False,None,None,31


In [48]:
user_diets['is_vegetarian'].value_counts()

True     114857
False    111713
Name: is_vegetarian, dtype: int64

In [49]:
user_diets['is_vegan'] = np.where(user_diets['vegan'] >= ((user_diets['recipe_totals'])*(.75)), True, False)
user_diets['is_vegan'].value_counts()

False    206565
True      20005
Name: is_vegan, dtype: int64

In [50]:
user_diets['is_GF'] = np.where(user_diets['GF'] >= ((user_diets['recipe_totals'])*(.75)), True, False)
user_diets['is_GF'].value_counts()

False    220978
True       5592
Name: is_GF, dtype: int64

In [51]:
user_diets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226570 entries, 0 to 226569
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   user_id        226570 non-null  int64 
 1   recipe_id      226570 non-null  object
 2   GF             226570 non-null  int64 
 3   vegetarian     226570 non-null  int64 
 4   vegan          226570 non-null  int64 
 5   is_vegetarian  226570 non-null  bool  
 6   is_vegan       226570 non-null  bool  
 7   is_GF          226570 non-null  bool  
 8   recipe_totals  226570 non-null  int64 
dtypes: bool(3), int64(5), object(1)
memory usage: 11.0+ MB


In [52]:
# tokenize ingredients:
#recipe_data['ingr_tokenized'] = recipe_data['ingredients'].apply(tokenizer.tokenize)
#recipe_data.head()

## Modeling

Collaborative Filtering approach - Model-Based
Matrix Factorization 

In [58]:
from surprise.prediction_algorithms.random_pred import NormalPredictor

from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD
from surprise.model_selection import GridSearchCV

In [59]:
print(user_df)

In [60]:
#NormalPredictor from surprise
# (Algorithm predicting a random rating based on the distribution of the training set, which is assumed to be normal.)
bsl_options = {'method': 'als',
               'n_epochs': 5,
               'reg_u': 12,
               'reg_i': 5
               }
trainset, testset = train_test_split(user_df, test_size=0.25)
algo_normal_predictor = NormalPredictor()
predictions = algo_normal_predictor.fit(trainset).test(testset)


In [61]:
norm_pred = cross_validate(NormalPredictor(), user_df, verbose=True)
print(np.mean(norm_pred['test_rmse']))

Evaluating RMSE, MAE of algorithm NormalPredictor on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.5888  1.5873  1.5858  1.5896  1.5826  1.5868  0.0025  
MAE (testset)     1.0793  1.0785  1.0758  1.0789  1.0746  1.0774  0.0019  
Fit time          1.51    2.12    2.10    2.06    2.03    1.96    0.23    
Test time         1.47    2.46    1.48    2.39    1.40    1.84    0.48    
1.586813533547491


In [66]:
#result = pd.DataFrame(predictions)
#result.drop(columns = {'details'}, inplace = True)
#result['error'] = abs(result['rating'] - result['predict_event'])
#result.head()

In [67]:
from surprise import BaselineOnly
#= BaselineOnly(bsl_options={}, verbose=True)

In [68]:
#BaselineOnly model
#(Algorithm predicting the baseline estimate for given user and item.)
cross_validate(BaselineOnly(), user_df, verbose=True)
#baseline = BaselineOnly(bsl_options={}, verbose=True)


Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating RMSE, MAE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.2182  1.2125  1.2115  1.2124  1.2099  1.2129  0.0028  
MAE (testset)     0.7395  0.7377  0.7370  0.7373  0.7361  0.7375  0.0011  
Fit time          5.88    6.09    6.04    6.06    6.20    6.05    0.10    
Test time         1.94    1.81    1.85    1.01    1.84    1.69    0.34    


{'test_rmse': array([1.21821355, 1.21246762, 1.2115064 , 1.2123942 , 1.20989049]),
 'test_mae': array([0.73946351, 0.73766577, 0.7369909 , 0.73727405, 0.73612373]),
 'fit_time': (5.875208139419556,
  6.088181972503662,
  6.038615942001343,
  6.057816982269287,
  6.199770927429199),
 'test_time': (1.937166690826416,
  1.8106451034545898,
  1.8451669216156006,
  1.0144178867340088,
  1.8439867496490479)}

In [ ]:
#SVD, SVDpp, NMF

In [73]:
## Perform a gridsearch with SVD
# ⏰ This cell may take several minutes to run
params = {'n_factors': [10, 20,50,75],
         'reg_all': [0.01, 0.02, 0.05]}
g_s_svd = GridSearchCV(SVD, param_grid=params, n_jobs=-1)
g_s_svd.fit(user_df)


In [74]:
# print out optimal parameters for SVD after GridSearch
print(g_s_svd.best_score)
print(g_s_svd.best_params)

{'rmse': 1.2128738683089053, 'mae': 0.7361912737657081}
{'rmse': {'n_factors': 10, 'reg_all': 0.05}, 'mae': {'n_factors': 10, 'reg_all': 0.01}}


In [69]:
from surprise import SVDpp

In [71]:
kf = KFold(n_splits=2) 
svd_pp = SVDpp()

In [55]:
svd_pp = cross_validate(SVDpp(), user_df, verbose=True)
print(np.svd_pp(norm_pred['test_rmse']))

KeyboardInterrupt: 

In [72]:
for train_set, test_set in kf.split(user_df):
    svd_pp.fit(trainset)
    svdpp_preds = svd_pp.test(testset) 
    accuracy.rmse(svdpp_preds, verbose=True)

NameError: name 'accuracy' is not defined

In [65]:
from surprise import NMF
from surprise.model_selection import KFold

kf = KFold(n_splits=3)
nmf_alg = NMF()

In [67]:
for train_set, test_set in kf.split(user_df):
    nmf_alg.fit(trainset)
    nmf_preds = nmf_alg.test(testset) 
    accuracy.rmse(nmf_preds, verbose=True)

ZeroDivisionError: float division

In [55]:
#need to determine best model
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD
#from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV

In [ ]:
#use best alg. with ALS (we have a large and sparse matrix ?)

print('Using ALS')
bsl_options = {'method': 'als',
               'n_epochs': 5,
               'reg_u': 12,
               'reg_i': 5
               }
algo = BaselineOnly(bsl_options=bsl_options)
cross_validate(algo, recipe_df, measures=['RMSE'], cv=3, verbose=False)


In [53]:



from surprise.prediction_algorithms import BaselineOnly
from surprise import accuracy 

bsl_options = {'method': 'als',
               'n_epochs': 5,
               'reg_u': 12,
               'reg_i': 5
               }

algo = BaselineOnly(bsl_options=bsl_options)
predictions = algo.fit(trainset).test(testset)
accuracy.rmse(predictions)

Estimating biases using als...
RMSE: 1.2132


1.2131928381465635

In [ ]:
# Create a dataframe with the cosine similarity scores
#similarity_df = pd.DataFrame(cosine_similarity, index=recipe_data['name'], columns=recipe_data['name'])

# Ask the user for a movie they like
#movie = input('Enter a movie you like: ')

# Find the index of the movie in the similarity dataframe
#movie_index = similarity_df.index.get_loc(movie)

# Get the top 10 most similar movies to the movie
#top_10 = similarity_df.iloc[movie_index].sort_values(ascending=False)[1:11]

# Print the top 10 most similar movies to the movie
#print(f'Top 10 similar movies to {movie}:')
#print(top_10)

In [ ]:
# train a model using the new combined DataFrame
#svd_ = SVD(n_factors=20, reg_all=0.1)
#svd_.fit(testset)
from surprise import accuracy

svd = SVD(n_factors=20, n_epochs=10, lr_all=0.005, reg_all=0.05)
svd.fit(trainset)
predictions = svd.test(testset)
print(accuracy.rmse(predictions))

____

In [ ]:
user_ratings = user_data.pivot(index='user_id', columns='recipe_id', values='rating')
user_ratings.head()

In [ ]:
for col in user_ratings:
    mean = user_ratings[col].mean()
    user_ratings[col] = user_ratings[col].fillna(value=mean)
user_ratings.head()

In [26]:
import numpy as np
import datetime

In [27]:
# ⏰ Expect this cell to take several minutes to run
start = datetime.datetime.now()
user_matrix = []
for i, row in enumerate(user_ratings.index):
    u1 = user_ratings[row]
    # Matrix is symetric, so fill in values for previously examined users
    user_distances = [entry[i] for entry in user_matrix] 
    for j, row2 in enumerate(user_ratings.index[i:]):
        u2 = user_ratings[row2]
        d = distance(u1,u2)
        user_distances.append(d)
    user_matrix.append(user_distances)
user_similarities = pd.DataFrame(user_matrix)

end = datetime.datetime.now()
elapsed = end - start
print(elapsed)

user_similarities.head()

NameError: name 'user_ratings' is not defined

In [ ]:
#function from rec. systems lesson, removes previously rated items and orders items for recommendations based on rating (high to low)
#based on avg of closest users ratings
def recommend_recipes(user, user_similarities, user_ratings, df, n_users=20, n_items=10):
    """n is the number of similar users who you wish to use to generate recommendations."""
    # User_Similarities Offset By 1 and Must Remove Current User
    top_n_similar_users = user_similarities[user-1].drop(user-1).sort_values().index[:n_users] 
    # Again, fixing the offset of user_ids
    top_n_similar_users = [i+1 for i in top_n_similar_users] 
    already_rated = set(df[df.user_id == 0].item_id.unique())
    unrated = set(df.item_id.unique()) - already_rated
    projected_user_ratings = user_ratings[user_ratings.index.isin(top_n_similar_users)].mean()[list(unrated)].sort_values(ascending=False)
    return projected_user_ratings[:n_items]

In [ ]:
recommend_movies(1, user_similarities, user_ratings, df)

In [ ]:
# return the top n recommendations using the 
def recommended_recipes(user_ratings,movie_title_df,n):
        for idx, rec in enumerate(user_ratings):
            title = movie_title_df.loc[movie_title_df['movieId'] == int(rec[0])]['title']
            print('Recommendation #', idx+1, ': ', title, '/n')
            n-=1 
            if n==0:
                break
                
recommended_movies(ranked_movies,df_movies,5)

- stemm or lemm text data
- 

limitation: person can be multiple 'diet types' at once